# MPI for python 

In this notebook, you will learn how to run your python code on many cores using Message Pasing Interface (MPI).  This is the type of "parallel computing" you should use to scale your code for running on "high performance computing" systems like Mies or Memex at carnegie. 


## What is parallel programming ?

If you run your ordinary python (or any other language) without any parallelization, in practice, it will use only 1 core of your machine while you have many more available. The purpose of parallelzation of a code is to use as many cores as possible to spped up your code. If you parallelize it in an efficient way, the speed-up will be roughly proportional to # of cores used. It is a great benifit when you have access to super-computers which have hunderds of cores avialable for you. 


Parallelize your code and get more work done !





## Quick review of a computer's structure :

A stand-alone computer like your current desktop is called a $\textbf{node}$ which contains few $\textbf{cores}$ (also clalled CPUs or $\textbf{processors}$). By default, the memory is shared among all these cores for the purpose of holding data while running your code. e.g. All cores know about the value of a variable which was calculated on any of the cores within the same node.

This picture illustrates 2 independant nodes, or computers, being connected via a network. In this example, each node contains 3 cores which share memory among themselves. On the other hand, the two separate nodes do not have shared memory. For example, if a core on the right node needs to know the value of a variable calculated on a core locacted on the other node, you need to send that data within the network connecting these two nodes. Therefore, communication between cores on different nodes are slower than cores within the same node.

![](./node-core.png)



## What are the options ?


There are 3 options to make your code parallel :

1. $\textbf{Simplest and fastests way}$ : If you want to run the same code on different data chunks, you can simply run your code multiple times and each time feed one chunck of the data to it. Each time you run your code, a single core is hookep up to run your code on it.


2. $\textbf{Shared-Memory parallelization}$: In this approach you let your machine distribute iterative commands like loops among the available cores. If you have 1000 iterations in a loop and 4 cores available, it runs 250 of them on each and at the end it will add up the results from each core to return a single final result. It is called $\textbf{shared memory}$ parallelization since the memory is physically shared among cores and you do not need to put any effort to communicate between them, i.e. they already know how to combine the results from each core to give you the final result. If you ever hear about $\textbf{threading}$ or $\textbf{Open-MP}$, it means this type of parallelization. The down side of this method is that you can never use more than a certain muber of cores (number of cores on a single node). Therefore, not the best option for running your code on clusters like Mies.

$\textbf{Bad News :}$ The structure of python language is not compatible with shared memory approach, therefore there is not any real implementation of this method for python. There are some work-arounds like [multi processing](https://docs.python.org/3/library/multiprocessing.html). Altough they are so easy to implement, it is not very natural to python and therefore so very limiting unlike the next method.


3. $\textbf{MPI}$ : It stands for $\textbf{Message Passing Interface}$ : In this case, you can run your code on as many as cores you want even if they are on different nodes. This is the only option when you want to use more than one node. By choosing number of processes (usually one process per core) lets call it np, machine runs a copy of your code on each processes and labels them by an integer called $\textbf{rank}$ (so number of ranks is the same as number of processes). Also, it establishes a $\textbf{communicator}$ to transfer in-memory data between different ranks. In MPI, none of the ranks share memory and therefore if you need them to communicate, you have to specify it in your code. 

Comparing MPI with shared memory or threaded method :

![0.2](./MPI-threaded-serial.png)


## Let's use MPI :

1.  First, you need to make sure you have MPI installed. List all the modules available on your system :

Mac:

```brew list```

Linux:

```module list```

If you find $\textbf{MPICH}$ on that list, you're good. If not, you should install it :

Mac:

``` brew install mpich ```

Linux (fedora) :

``` dnf install  mpich ```

Linux (Ubuntu/Debian) :

``` sudo apt install mpich ```

2. Fortunately a stable MPI package is avaiable for python, [mpi4py](https://mpi4py.readthedocs.io/en/stable/). Install this via conda :

``` conda install -c anaconda mpi4py ```

Now we are ready to go !


### Hello World example :

The simplest example is asking each rank to print their rank id. Type/copy the code below in a file named `hello_world.py` :

In [1]:
import mpi4py
from mpi4py import MPI

# The communicator among ranks
comm = MPI.COMM_WORLD
# Total number of ranks
size = comm.Get_size()
rank = comm.Get_rank()

print('Hi, I am rank ', rank, ' among ', size, ' available ranks!')

if rank == 0:
    print('Rank ', rank, ' : You can command me privately by specifying my id !')


Hi, I am rank  0  among  1  available ranks!
Rank  0  : You can command me privately by specifying my id !



Then, you need to run your script on terminal like :

` mpirun -np 2 python hellow_world.py`

where `-np 2` specifies number of processors you want to use. To see how many cores you have on any Unix machine you can use `htop` package. Type `htop` on terminal and see if you have it installed. If not, you can install it with :

mac :

` brew install htop`

Linux (fedora) :

`dnf install htop`

Linux (Ubunto/Debian)

` sudo apt install htop`

Here is a snapshot of what you're expected to see. It shows my eesktop has 12 cores and none of them are busy.

![htop](htop.png)


## Comunicating among ranks :


MPI has so many buit-in functions to help you communicate among ranks. There are 2 general type of communications :

### Ponit-to-point communications :

Sometimes you need to send a scalar, numpy arrary or a python object from one rank to another rank. It can be done with functions below :

- `send() and recieve()` : For transferring scalars and python objects

- `Send() and Recieve()` : For transfering numpy arrays


#### Example : 

(Credit : [mpi4py documentaion](https://mpi4py.readthedocs.io/en/stable/tutorial.html#point-to-point-communication) )

Python objects:

In [ ]:
import mpi4py
from mpi4py import MPI

# The communicator among ranks
comm = MPI.COMM_WORLD
# Total number of ranks
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    comm.send(data, dest=1, tag=11)
elif rank == 1:
    data = comm.recv(source=0, tag=11)
    print('rank ', rank, 'data = ', data)

NumPy arrays (the fast way!) :

In [ ]:
import mpi4py
from mpi4py import MPI
import numpy as np

# The communicator among ranks
comm = MPI.COMM_WORLD
# Total number of ranks
size = comm.Get_size()
rank = comm.Get_rank()

# passing MPI datatypes explicitly
if rank == 0:
    data = np.arange(10, dtype='i')
    comm.Send([data, MPI.INT], dest=1, tag=77)
elif rank == 1:
    # You need a buffer for the array
    data = np.empty(10, dtype='i')
    comm.Recv([data, MPI.INT], source=0, tag=77)
    print('Rank ', rank, ' data ', data)

# automatic MPI datatype discovery
if rank == 0:
    data = np.arange(10, dtype=np.float64)
    comm.Send(data, dest=1, tag=13)
elif rank == 1:
    data = np.empty(10, dtype=np.float64)
    comm.Recv(data, source=0, tag=13)
    print('Rank ', rank, ' data. ', data)


 - Tip : All functions for communicating Numpy arrays srtart with capital letters. like `Send()` and `Recieve()`.

### Collective communications :

On this type of communication, one rank can sned/recieve to/from all ranks. So, all ranks are involved. 

Examples are :

` bcast(), Bcast()` : Send a python object or numpy array from one rank to all other ranks :

` scatter(), Scatter()` : Scatter a list or numpy array among all ranks 

`  gather(), Gather() ` : Gather obejcts or numpy arrays in a single list or numpy array


#### Examples :

(Credit : [mpi4py documentaion](https://mpi4py.readthedocs.io/en/stable/tutorial.html#point-to-point-communication) )

Broadcasting for python objects:

In [3]:
import mpi4py
from mpi4py import MPI

# The communicator among ranks
comm = MPI.COMM_WORLD
# Total number of ranks
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    data = {'key1' : [7, 2.72, 2+3j],
            'key2' : ( 'abc', 'xyz')}
else:
    # You need a buffer
    data = None
data = comm.bcast(data, root=0)
print('Rank :', rank, 'data =', data)

Rank : 0 data = {'key1': [7, 2.72, (2+3j)], 'key2': ('abc', 'xyz')}


Scattering for python objects:

In [5]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    data = [(i+1)**2 for i in range(size)]
else:
    data = None

data = comm.scatter(data, root=0)
print('Rank ', rank, 'data = ', data)
assert data == (rank+1)**2

1


Gathering python objects:

In [2]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

data = (rank+1)**2
data = comm.gather(data, root=0)

    
print('Rank', rank, 'data =', data)

Rank 0 data = [1]


Numpy arrays :

In [5]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    data = np.arange(10, dtype='i')
    # You need to make sure the array's elements are contiguius in memory
    data = np.ascontiguousarray(data)

else:
    # Make a buffer for array
    data = np.empty(10, dtype='i')
    # You need to make sure the array's elements are contiguius in memory
    data = np.ascontiguousarray(data)

comm.Bcast(data, root=0)
for i in range(10):
    assert data[i] == i
print('Rank ', rank, ' data ', data)

In [ ]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

sendbuf = None
if rank == 0:
    sendbuf = np.empty([size, 10], dtype='i')
    sendbuf.T[:,:] = range(size)
    # You need to make sure the array's elements are contiguius in memory
    sendbuf = np.ascontiguousarray(sendbuf)

recvbuf = np.empty(10, dtype='i')
# You need to make sure the array's elements are contiguius in memory
recvbuf = np.ascontiguousarray(recvbuf)

print('Rank ', rank, ' sendbuf ', sendbuf)
comm.Scatter(sendbuf, recvbuf, root=0)
print('Rank ', rank, ' recvbuf ', recvbuf)
assert np.allclose(recvbuf, rank)

In [ ]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

sendbuf = np.zeros(10, dtype='i') + rank
# You need to make sure the array's elements are contiguius in memory
sendbuf = np.ascontiguousarray(sendbuf)
print('Rank ', rank, ' sendbuf ' , sendbuf)
recvbuf = None
if rank == 0:
    recvbuf = np.empty([size, 10], dtype='i')
    # You need to make sure the array's elements are contiguius in memory
    recvbuf = np.ascontiguousarray(recvbuf)
comm.Gather(sendbuf, recvbuf, root=0)

print('Rank ', rank, ' recvbuf ' , recvbuf)

In [ ]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()


data = np.arange(10)*0.1*rank
# You need to make sure the array's elements are contiguius in memory
data = np.ascontiguousarray(data)
print('Rank ', rank, ' data before allreduce ', data)
comm.Allreduce(MPI.IN_PLACE, data, op=MPI.SUM)
print('Rank ', rank, ' data after  allreduce ', data)

- There are many other built-in functions that are optimized for minimal computaion and memory usage. [This website](https://pages.tacc.utexas.edu/~eijkhout/pcse/html/index.html) is a great resource for learning more about MPI and all other useful communications. These functions behave similarly in python, C and Fortran, but the syntax is slightly different in each. This website has examples for all 3  languages on each topic. 

- I also recommend you to look at [mpi4py documentaion](https://mpi4py.readthedocs.io/en/stable/tutorial.html#point-to-point-communication) and any other resource you might found on google.